In [1]:

import geopandas as gpd
import momepy
import networkx as nx
import graph_tool.all as gt
from scripts.nx2gt import nx2gt
import pandas as pd
import matplotlib
import itertools
from shapely.geometry import LineString, point
from shapely.wkt import loads
import geopandas as gpd
from tqdm import tqdm
import numpy as np
from dask.distributed import Client, LocalCluster
from dask import delayed
import dask.dataframe as dd
import dask
import math

/tmp/ipykernel_3569/3823283595.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

2023-04-16 22:56:47,060 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-16 22:56:47,063 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-16 22:56:47,065 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-16 22:56:47,067 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-16 22:56:47,070 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-16 22:56:47,072 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-16 22:56:47,076 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-16 22:56:47,

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35287,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:39819,Total threads: 2
Dashboard: http://127.0.0.1:43413/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:36999,


In [ ]:
streets = gpd.read_parquet("./out/singapore/streets.pq").explode()

streets['edgeID'] = momepy.unique_id(streets)

In [ ]:
# Create a pandas dataframe from the multigraph
primal = momepy.gdf_to_nx(streets, length = "mm_len", approach='primal')

In [ ]:
vertID_dict = {}

In [ ]:
counter = itertools.count()

for node in primal.nodes():
    id = next(counter)
    primal.nodes[node]['vertID'] = id
    vertID_dict[id] = node

In [ ]:
inverted_vertID_dict = {value: key for key, value in vertID_dict.items()}

In [ ]:
primal_scattered = client.scatter(primal)

@delayed
def compute_ego_graph(n, primal_scattered, radius):
    graph = nx2gt(nx.ego_graph(primal_scattered, n, radius, distance="mm_len"))
    return (n, graph)

tasks = []
for n in primal.nodes():
    task = compute_ego_graph(n, primal_scattered, 400)
    tasks.append(task)
    
results = dask.compute(*tasks)

In [ ]:
ego_graphs = {}

for n, ego_graph in results:
    ego_graphs[inverted_vertID_dict[n]] = {}
    # 
    ego_graphs[inverted_vertID_dict[n]]["graph"] = ego_graph

    if len(list(ego_graph.edges())) > 1:
        ego_graphs[inverted_vertID_dict[n]]["weight"] = ego_graph.edge_properties["mm_len"]
    else:
        ego_graphs[inverted_vertID_dict[n]]["weight"] = None
        
    ego_graphs[inverted_vertID_dict[n]]["node"] = ego_graph.get_vertices()[[i for i,v in enumerate(ego_graph.vp.vertID.get_array()) if v == inverted_vertID_dict[n]][0]]

In [ ]:
# Convert it to a graph-tool graph
gtG = nx2gt(primal)

gtG.list_properties()

In [ ]:
g = gt.GraphView(gtG, vfilt=gt.label_largest_component(gtG))
mm_len = gtG.edge_properties["mm_len"]
vp, ep = gt.betweenness(g, weight = mm_len)

In [ ]:
edgeID = g.ep.edgeID.get_array()

betweeness_centrality = ep.get_array()

geometry = g.ep.geometry.get_array()

edges_frame = pd.merge(pd.DataFrame({'edgeID': edgeID, 'betweeness_centrality': betweeness_centrality, 'geometry': geometry}), streets, on='edgeID').drop(columns=["geometry_x"]).rename(columns={"geometry_y": "geometry"})
edges_frame = gpd.GeoDataFrame(edges_frame, geometry=edges_frame['geometry'], crs=streets.crs)
edges_frame = edges_frame.sort_values(by=['edgeID'], ascending=True).reset_index(drop=True)
edges_frame

In [ ]:
vertex_id = g.vp.vertID.get_array()

vp_array = vp.get_array()

vertices_frame = pd.DataFrame({'vertID': vertex_id, 'betweeness_centrality': vp_array})
# convert vertID_dict to a DataFrame and rename the index and column
vertID_df = pd.DataFrame.from_dict(vertID_dict, orient='index').rename(columns={0: "y", 1: "x"})
# merge vertID_df and vertices on vertID and node, respectively
vertices_frame = vertID_df.merge(vertices_frame, left_index=True, right_on='vertID')
vertices_frame["geometry"] = vertices_frame.apply(lambda x: point.Point(x["x"], x["y"]), axis=1)
vertices_frame = vertices_frame.drop(columns=["x", "y"])


In [ ]:
node_degree = {g.vertex_index[v]: v.in_degree() + v.out_degree() for v in g.vertices()}
# convert vertID_dict to a DataFrame and rename the index and column
node_degree = pd.DataFrame.from_dict(node_degree, orient='index').rename(columns={0: "node_degree"})
# merge vertID_df and vertices on vertID and node, respectively
vertices_frame = vertices_frame.merge(node_degree, right_index=True, left_on='vertID')

In [ ]:
closeness = {k:gt.closeness(ego_graphs[k]["graph"], weight=ego_graphs[k]["weight"], source=ego_graphs[k]["node"]) for k in tqdm(ego_graphs)}
closeness_df = pd.DataFrame({'closeness': closeness})
closeness_df = closeness_df.applymap(lambda x: x.astype(float)).fillna(0)
vertices_frame = vertices_frame.merge(closeness_df, right_index=True, left_on='vertID')

In [ ]:
global_clustering_400m = {k:float(gt.global_clustering(ego_graphs[k]["graph"])[0]) for k in tqdm(ego_graphs)}
global_clustering_400m  = pd.DataFrame.from_dict(global_clustering_400m , orient='index').rename(columns={0: "global_clustering_400m"})
vertices_frame = vertices_frame.merge(global_clustering_400m, right_index=True, left_on='vertID')

In [ ]:
import pickle

# your code to create gtG and vertID_dict goes here

# save gtG and vertID_dict to a file using pickle
with open('data.pkl', 'wb') as f:
    pickle.dump((gtG, vertID_dict), f)

In [5]:
import pickle
# load the data from the file
with open('data.pkl', 'rb') as f:
    gtG, vertID_dict = pickle.load(f)

In [6]:
def euclidean_dist(x1, y1, x2, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def bravo(target, n, vertID_dict_scattered, network_dist):
    euclidean_distance = euclidean_dist(vertID_dict_scattered[n][0], vertID_dict_scattered[n][1], vertID_dict_scattered[target][0], vertID_dict_scattered[target][1])
    return euclidean_distance / network_dist

@dask.delayed
def alpha(G, n, vertID_dict_scattered):
    straightness = 0
    sp = gt.shortest_distance(G, n, weights=G.edge_properties["mm_len"])
    sp_scattered = sp

    if len(sp.get_array()) > 0 and len(G) > 1:
        for target, value in enumerate(sp):
            if n != target:
                network_dist = sp_scattered[target]
                straightness += bravo(target, n, vertID_dict_scattered, network_dist)
        straightness_df = straightness * (1.0 / (len(vertID_dict_scattered.keys()) - 1.0))
    else:
        straightness_df = 0
    
    return n, straightness_df

# def straightness_centrality(G, vertID_dict):
#     G_scattered = client.scatter(G)
#     vertID_dict_scattered = client.scatter(vertID_dict)

#     results = []
#     delayed_objs=[]
#     i=0
#     for n in G.iter_vertices():
        
#         delayed_objs=[]
#         delayed_objs.append(alpha(G_scattered, n, vertID_dict_scattered))
    
#     chunk_size = 8
#     delayed_objs_chunks = [delayed_objs[i:i+chunk_size] for i in range(0, len(delayed_objs), chunk_size)]
    
#     for chunk in delayed_objs_chunks:
#         new_results = client.compute(chunk)
#         results.append(new_results)
#         print(new_results)
    
#     return results
    
    

def straightness_centrality(G, vertID_dict):
    G_scattered = client.scatter(G)
    vertID_dict_scattered = client.scatter(vertID_dict)

    delayed_objs=[]
    for n in G.iter_vertices():
        delayed_objs.append(dask.delayed(alpha)(G_scattered, n, vertID_dict_scattered))

    results = dask.compute(*delayed_objs)
    return results

In [7]:
straightness_df = straightness_centrality(gtG, vertID_dict)

2023-04-16 22:58:28,705 - distributed.utils_perf - WARNING - full garbage collections took 32% CPU time recently (threshold: 10%)
2023-04-16 22:58:29,886 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2023-04-16 22:58:31,105 - distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)
2023-04-16 22:58:32,302 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2023-04-16 22:58:33,514 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2023-04-16 22:58:34,740 - distributed.utils_perf - WARNING - full garbage collections took 36% CPU time recently (threshold: 10%)
2023-04-16 22:58:35,975 - distributed.utils_perf - WARNING - full garbage collections took 36% CPU time recently (threshold: 10%)
2023-04-16 22:58:37,209 - distributed.utils_perf - WARNING - full garbage collections took

In [ ]:
daskCluster.close()
client.shutdown()
